# Design Patterns with First-Class Functions
Although design patterns are language-independent, that does not mean every pattern aplies to every language. In particular, in the context of languages with first-class functions, Norvig suggests rethinking the Strategy, Command, Template Method, and Visitor patterns. 

## The Strategy pattern
In layman's terms, a Design Pattern describes a problem and a general approach to solving it. The strategy pattern (aka 'The Policy Pattern') is one of the most frequently used Behavioral Pattern out there. It is also one of the simplest. See [this notebook](https://github.com/simongarisch/Python-Design-Patterns/blob/master/15%20The%20Strategy%20Pattern.ipynb) on the strategy pattern.

## Case Study: Refactoring Strategy
Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it. strategy participants are:

Context:
    Provides a service by delegating some computation to interchangeable components
    that implement alternative algorithms. In the ecommerce example, the context is
    an Order, which is configured to apply a promotional discount according to one of
    several algorithms.
    
Strategy:
    The interface common to the components that implement the different algorithms.
    In our example, this role is played by an abstract class called Promotion.
    
Concrete Strategy:
    One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo, and Large
    OrderPromo are the three concrete strategies implemented.

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


In [2]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


In [5]:
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


In [6]:
class Promotion(ABC): # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


In [7]:
class FidelityPromo(Promotion): # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0


In Python 3.4, the simplest way to declare an ABC is to subclass abc.ABC, as I did in Example 6-1. From Python 3.0 to 3.3, you must use the metaclass= keyword in the class statement (e.g., class
Promotion(metaclass=ABCMeta):).

In [9]:
# Two customers: joe has 0 fidelity points, ann has 1,100
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

# One shopping cart with three line items.
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

# The FidelityPromo promotion gives no discount to joe.
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [11]:
# ann gets a 5% discount because she has at least 1,000 points.
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

## Function-Oriented Strategy
Here we are replacing the concrete strategies with simple functions and removing the Promo abstract class.

In [12]:
from collections import namedtuple
Customer = namedtuple('Customer', 'name fidelity')

def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0


Rather than rewrite the order class with a new 'due' method I'm going to overwrite this:

In [16]:
class NewOrder(Order):
    '''
    Rather than call a discount method of a class
    we call a proomotion function which takes the order
    object as an argument.
    '''
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount


In [17]:
# Two customers: joe has 0 fidelity points, ann has 1,100
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

# One shopping cart with three line items.
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

# The FidelityPromo promotion gives no discount to joe.
NewOrder(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [18]:
# ann gets a 5% discount because she has at least 1,000 points.
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

## Choosing the Best Strategy: Simple Approach